A toy demo to train a GPT on a sorting problem.

In [1]:
import torch
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from mingpt.utils import set_seed
set_seed(3407)

In [2]:
import pickle

class SortDataset(Dataset):
    """ 
    Dataset for the Sort problem. E.g. for problem length 6:
    Input: 0 0 2 1 0 1 -> Output: 0 0 0 1 1 2
    Which will feed into the transformer concatenated as:
    input:  0 0 2 1 0 1 0 0 0 1 1
    output: I I I I I 0 0 0 1 1 2
    where I is "ignore", as the transformer is reading the input sequence
    """

    def __init__(self, split, length=6, num_digits=3):
        assert split in {'train', 'test'}
        self.split = split
        self.length = length
        self.num_digits = num_digits
    
    def __len__(self):
        return 10000
    
    def get_vocab_size(self):
        return self.num_digits
    
    def get_block_size(self):
        # the length of the sequence that will feed into transformer, 
        # containing concatenated input and the output, but -1 because
        # the transformer starts making predictions at the last input element
        return self.length * 2 - 1

    def __getitem__(self, idx):
        # use rejection sampling to generate an input example from the desired split
        while True:
            # generate some random integers
            inp = torch.randint(self.num_digits, size=(self.length,), dtype=torch.long)
            # half of the time let's try to boost the number of examples that 
            # have a large number of repeats, as this is what the model seems to struggle
            # with later in training, and they are kind of rate
            if torch.rand(1).item() < 0.5:  # 50% chance
                if inp.unique().nelement() > self.length // 2:
                    # too many unqiue digits, re-sample
                    continue
            # figure out if this generated example is train or test based on its hash
            h = hash(pickle.dumps(inp.tolist()))
            inp_split = 'test' if h % 4 == 0 else 'train' # designate 25% of examples as test
            if inp_split == self.split:
                break # ok

        # solve the task: i.e. sort
        sol = torch.sort(inp)[0]

        # concatenate the problem specification and the solution
        cat = torch.cat((inp, sol), dim=0)

        # the inputs to the transformer will be the offset sequence
        x = cat[:-1].clone()
        y = cat[1:].clone()
        # we only want to predict at output locations, mask out the loss at the input locations
        y[:self.length-1] = -1  # i.e. ignore
        return x, y

In [3]:
# print an example instance of the dataset
train_dataset = SortDataset('train', length=6, num_digits=10)
test_dataset = SortDataset('test', length=6, num_digits=10)
x, y = train_dataset[0]
xline = [f"{a:2d}" for a in x]
yline = [f"{a:2d}" for a in y]
print(" ".join(xline))
print(" ".join(yline))

 9  8  1  9  1  0  0  1  1  8  9
-1 -1 -1 -1 -1  0  1  1  8  9  9


In [4]:
# create a GPT instance
from mingpt.model import GPT

model_config = GPT.get_default_config()
model_config.model_type = 'gpt-mini'
model_config.vocab_size = train_dataset.get_vocab_size()
model_config.block_size = train_dataset.get_block_size()
model = GPT(model_config)

number of parameters: 2.67M


In [10]:
# create a Trainer object
from mingpt.trainer import Trainer

train_config = Trainer.get_default_config()
train_config.learning_rate = 5e-4 # the model we're using is so small that we can go a bit faster
train_config.max_iters = 1001
train_config.num_workers = 0
trainer = Trainer(train_config, model, train_dataset)
print(trainer.device)

running on device cuda
cuda


In [11]:
def batch_end_callback(trainer):
    if trainer.iter_num % 200 == 0:
        print(f"iter_dt {trainer.iter_dt * 1000:.2f}ms; iter {trainer.iter_num}: train loss {trainer.loss.item():.5f}")
trainer.set_callback('on_batch_end', batch_end_callback)

trainer.run()

iter_dt 0.00ms; iter 0: train loss 0.01402
iter_dt 14.51ms; iter 200: train loss 0.04656
iter_dt 14.50ms; iter 400: train loss 0.01239
iter_dt 14.92ms; iter 600: train loss 0.02817
iter_dt 14.81ms; iter 800: train loss 0.01273
iter_dt 14.59ms; iter 1000: train loss 0.01084


In [12]:
# now let's perform some evaluation
model.eval()

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(10, 192)
    (wpe): Embedding(11, 192)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x Block(
        (ln_1): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=192, out_features=576, bias=True)
          (c_proj): Linear(in_features=192, out_features=192, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
        (mlp): ModuleDict(
          (c_fc): Linear(in_features=192, out_features=768, bias=True)
          (c_proj): Linear(in_features=768, out_features=192, bias=True)
          (act): NewGELU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_feature

In [13]:
def eval_split(trainer, split, max_batches):
    dataset = {'train':train_dataset, 'test':test_dataset}[split]
    n = train_dataset.length # naugy direct access shrug
    print(f"dataset length: {n}")
    print(f"dataset size: train={len(train_dataset)} test={len(test_dataset)}")
    results = []
    mistakes_printed_already = 0
    loader = DataLoader(dataset, batch_size=100, num_workers=0, drop_last=False)
    for b, (x, y) in enumerate(loader):
        x = x.to(trainer.device)
        y = y.to(trainer.device)
        # isolate the input pattern alone
        inp = x[:, :n]
        sol = y[:, -n:]
        # let the model sample the rest of the sequence
        cat = model.generate(inp, n, do_sample=False) # using greedy argmax, not sampling
        if b == 0:
            print(f"input={inp[0].tolist()} solution={sol[0].tolist()}")
            print(f"raw output={cat[0].tolist()}")
        sol_candidate = cat[:, n:] # isolate the filled in sequence
        # compare the predicted sequence to the true sequence
        correct = (sol == sol_candidate).all(1).cpu() # Software 1.0 vs. Software 2.0 fight RIGHT on this line haha
        for i in range(x.size(0)):
            results.append(int(correct[i]))
            if not correct[i] and mistakes_printed_already < 3: # only print up to 5 mistakes to get a sense
                mistakes_printed_already += 1
                print("GPT claims that %s sorted is %s but gt is %s" % (inp[i].tolist(), sol_candidate[i].tolist(), sol[i].tolist()))
        if max_batches is not None and b+1 >= max_batches:
            break
    rt = torch.tensor(results, dtype=torch.float)
    print("%s final score: %d/%d = %.2f%% correct" % (split, rt.sum(), len(results), 100*rt.mean()))
    return rt.sum()

# run a lot of examples from both train and test through the model and verify the output correctness
with torch.no_grad():
    train_score = eval_split(trainer, 'train', max_batches=50)
    test_score  = eval_split(trainer, 'test',  max_batches=50)

dataset length: 6
dataset size: train=10000 test=10000
input=[9, 8, 7, 5, 1, 9] solution=[1, 5, 7, 8, 9, 9]
raw output=[9, 8, 7, 5, 1, 9, 1, 5, 7, 8, 9, 9]
GPT claims that [8, 5, 5, 5, 8, 5] sorted is [5, 5, 5, 5, 5, 8] but gt is [5, 5, 5, 5, 8, 8]
GPT claims that [2, 3, 2, 6, 4, 6] sorted is [2, 2, 4, 4, 6, 6] but gt is [2, 2, 3, 4, 6, 6]
GPT claims that [4, 2, 4, 2, 7, 3] sorted is [2, 3, 4, 4, 4, 7] but gt is [2, 2, 3, 4, 4, 7]
train final score: 4971/5000 = 99.42% correct
dataset length: 6
dataset size: train=10000 test=10000
input=[0, 4, 3, 4, 0, 8] solution=[0, 0, 3, 4, 4, 8]
raw output=[0, 4, 3, 4, 0, 8, 0, 0, 3, 4, 4, 8]
GPT claims that [6, 3, 4, 2, 4, 2] sorted is [2, 3, 4, 4, 4, 6] but gt is [2, 2, 3, 4, 4, 6]
GPT claims that [8, 9, 9, 9, 8, 9] sorted is [9, 9, 9, 9, 9, 9] but gt is [8, 8, 9, 9, 9, 9]
GPT claims that [4, 2, 7, 2, 4, 2] sorted is [2, 2, 4, 4, 4, 7] but gt is [2, 2, 2, 4, 4, 7]
test final score: 4972/5000 = 99.44% correct


In [14]:
# let's run a random given sequence through the model as well
n = train_dataset.length # naugy direct access shrug
inp = torch.tensor([[0, 0, 2, 1, 0, 1]], dtype=torch.long).to(trainer.device)
assert inp[0].nelement() == n
with torch.no_grad():
    cat = model.generate(inp, n, do_sample=False)
sol = torch.sort(inp[0])[0]
sol_candidate = cat[:, n:]
print('input sequence  :', inp.tolist())
print('predicted sorted:', sol_candidate.tolist())
print('gt sort         :', sol.tolist())
print('matches         :', bool((sol == sol_candidate).all()))

input sequence  : [[0, 0, 2, 1, 0, 1]]
predicted sorted: [[0, 0, 0, 1, 1, 2]]
gt sort         : [0, 0, 0, 1, 1, 2]
matches         : True


In [21]:
# let's try a bad example with longer input sequence length
n = train_dataset.length
inp = torch.tensor([[8, 0, 2, 0, 1, 3]], dtype=torch.long).to(trainer.device)
with torch.no_grad():
    cat = model.generate(inp, n + 2, do_sample=False)
sol = torch.sort(inp[0])[0]
sol_candidate = cat[:, n:]
print('input sequence  :', inp.tolist())
print('predicted sorted:', sol_candidate.tolist())
print('gt sort         :', sol.tolist())

input sequence  : [[8, 0, 2, 0, 1, 3]]
predicted sorted: [[0, 0, 1, 2, 3, 8, 8, 8]]
gt sort         : [0, 0, 1, 2, 3, 8]
